<a href="https://colab.research.google.com/github/Adhi1755/Power-Consumption-Demand-Forecasting/blob/main/Mini_Project_on_Demand_Forecasting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **Demand Forecasting on Power Consumption**

## **Importing Required Libraries**

In [7]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

## **Basic Analysis on the Data**

In [8]:
df = pd.read_csv("/content/drive/MyDrive/Google Colab/Mini Project/household_power_consumption.csv")

/tmp/ipython-input-2250291733.py:1: DtypeWarning: Columns (2,3,4,5,6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/content/drive/MyDrive/Google Colab/Mini Project/household_power_consumption.csv")


In [9]:
df.head()

,Date,Time,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
0,16/12/2006,17:24:00,4.216,0.418,234.840,18.400,0.000,1.000,17.0
1,16/12/2006,17:25:00,5.360,0.436,233.630,23.000,0.000,1.000,16.0
2,16/12/2006,17:26:00,5.374,0.498,233.290,23.000,0.000,2.000,17.0
3,16/12/2006,17:27:00,5.388,0.502,233.740,23.000,0.000,1.000,17.0
4,16/12/2006,17:28:00,3.666,0.528,235.680,15.800,0.000,1.000,17.0


In [10]:
df.shape

(2075259, 9)

In [11]:
df.describe()

,Sub_metering_3
count,2.049280e+06
mean,6.458447e+00
std,8.437154e+00
min,0.000000e+00
25%,0.000000e+00
50%,1.000000e+00
75%,1.700000e+01
max,3.100000e+01


## **Ensuring Correct Data Types for Analysis**

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075259 entries, 0 to 2075258
Data columns (total 9 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   Date                   object 
 1   Time                   object 
 2   Global_active_power    object 
 3   Global_reactive_power  object 
 4   Voltage                object 
 5   Global_intensity       object 
 6   Sub_metering_1         object 
 7   Sub_metering_2         object 
 8   Sub_metering_3         float64
dtypes: float64(1), object(8)
memory usage: 142.5+ MB


In [13]:
numeric_cols = [
    "Global_active_power",
    "Global_reactive_power",
    "Voltage",
    "Global_intensity",
    "Sub_metering_1",
    "Sub_metering_2"
]

for col in numeric_cols:
    df[col] = pd.to_numeric(df[col], errors="coerce")


In [14]:
df["Date"] = pd.to_datetime(df["Date"], format="%d/%m/%Y")

In [15]:
df["Datetime"] = pd.to_datetime(
    df["Date"].astype(str) + " " + df["Time"]
)

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075259 entries, 0 to 2075258
Data columns (total 10 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   Date                   datetime64[ns]
 1   Time                   object        
 2   Global_active_power    float64       
 3   Global_reactive_power  float64       
 4   Voltage                float64       
 5   Global_intensity       float64       
 6   Sub_metering_1         float64       
 7   Sub_metering_2         float64       
 8   Sub_metering_3         float64       
 9   Datetime               datetime64[ns]
dtypes: datetime64[ns](2), float64(7), object(1)
memory usage: 158.3+ MB


In [17]:
df.dtypes

,0
Date,datetime64[ns]
Time,object
Global_active_power,float64
Global_reactive_power,float64
Voltage,float64
Global_intensity,float64
Sub_metering_1,float64
Sub_metering_2,float64
Sub_metering_3,float64
Datetime,datetime64[ns]


In [18]:
df.head()

,Date,Time,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3,Datetime
0,2006-12-16,17:24:00,4.216,0.418,234.84,18.4,0.0,1.0,17.0,2006-12-16 17:24:00
1,2006-12-16,17:25:00,5.360,0.436,233.63,23.0,0.0,1.0,16.0,2006-12-16 17:25:00
2,2006-12-16,17:26:00,5.374,0.498,233.29,23.0,0.0,2.0,17.0,2006-12-16 17:26:00
3,2006-12-16,17:27:00,5.388,0.502,233.74,23.0,0.0,1.0,17.0,2006-12-16 17:27:00
4,2006-12-16,17:28:00,3.666,0.528,235.68,15.8,0.0,1.0,17.0,2006-12-16 17:28:00


In [19]:
df.isnull().sum()

,0
Date,0
Time,0
Global_active_power,25979
Global_reactive_power,25979
Voltage,25979
Global_intensity,25979
Sub_metering_1,25979
Sub_metering_2,25979
Sub_metering_3,25979
Datetime,0


In [20]:
df.duplicated().sum()

np.int64(0)

In [21]:
df[df.isnull().any(axis=1)]

,Date,Time,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3,Datetime
6839,2006-12-21,11:23:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2006-12-21 11:23:00
6840,2006-12-21,11:24:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2006-12-21 11:24:00
19724,2006-12-30,10:08:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2006-12-30 10:08:00
19725,2006-12-30,10:09:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2006-12-30 10:09:00
41832,2007-01-14,18:36:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2007-01-14 18:36:00
...,...,...,...,...,...,...,...,...,...,...
1990185,2010-09-28,19:09:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2010-09-28 19:09:00
1990186,2010-09-28,19:10:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2010-09-28 19:10:00
1990187,2010-09-28,19:11:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2010-09-28 19:11:00
1990188,2010-09-28,19:12:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2010-09-28 19:12:00


In [22]:
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)

In [23]:
df.isnull().sum()

,0
Date,0
Time,0
Global_active_power,0
Global_reactive_power,0
Voltage,0
Global_intensity,0
Sub_metering_1,0
Sub_metering_2,0
Sub_metering_3,0
Datetime,0


In [24]:
df.shape

(2049280, 10)

## **Time Gap Analysis in the Time Series Data**

As an initial step, we check the time range of the dataset using the **min() and max()** values of the Datetime column to understand the total duration covered by the data.



In [26]:
df['Datetime'].min(), df['Datetime'].max()

(Timestamp('2006-12-16 17:24:00'), Timestamp('2010-11-26 21:02:00'))

Since this is time series data, analyzing temporal consistency is essential because most time series models assume observations occur at regular intervals. To identify irregularities, we use the **diff()** function on the Datetime column, which computes the time difference between consecutive timestamps.



In [27]:
df['Datetime'].diff().value_counts().head()

,count
Datetime,
0 days 00:01:00,2049208
0 days 00:02:00,38
0 days 00:03:00,14
0 days 00:04:00,2
2 days 14:04:00,1


The analysis shows that the dataset is largely consistent, with most observations recorded at a 1-minute interval, making it well-suited for time series analysis and forecasting. A small number of irregular gaps are present: **2-minute gaps (38 rows)**, **3-minute gaps (14 rows)**, and **4-minute gaps (2 rows**). These minor gaps are typical in real-world sensor data and can be handled using resampling and interpolation.

Additionally, **a single large gap of approximately 2 days is observed**, which is critical and likely indicates a data collection interruption. This gap requires special handling, as it may impact trend analysis and forecasting performance.

Overall, the dataset exhibits strong temporal consistency, and appropriate gap-handling techniques will be applied to prepare it for reliable time series modeling.

In [28]:
df[['Global_active_power',
    'Voltage',
    'Global_intensity']].skew()


,0
Global_active_power,1.786233
Voltage,-0.326665
Global_intensity,1.849100
